In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df=spark.read.format("delta").option("inferSchema",True).option("header",True).load("abfss://bronze@netflixstorageacct.dfs.core.windows.net/netflix_titles")


In [0]:
display(df)

In [0]:
df=df.fillna({'duration_minutes':0,'duration_seasons':1})
df.display()

In [0]:
df=df.withColumn("title",split(col("title"),":")[0])
display(df)


In [0]:
df=df.withColumn("rating",split(col("rating"),"-")[0])
display(df)

In [0]:
df=df.filter(col("duration_minutes") != 'Flying Fortress""')
display(df)

In [0]:
from pyspark.sql.functions import when, col
df=df.withColumn("is_movie", when(col("type") == "Movie", 1).when(col("type") == "TV Show", 2).otherwise(0))


In [0]:
df=df.withColumn("duration_rank",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))
df.display()

In [0]:
df.write.format("delta").mode("overwrite")\
        .option("path","abfss://silver@netflixstorageacct.dfs.core.windows.net/netflix_titles").save()